In [7]:
import os, random, csv, datetime, json
import pandas as pd
import numpy as np
import argparse
import time
from tqdm import tqdm

In [8]:
def gen_feats(task,head,link,cross,nl):
########################
###head features
########################
    head['speed'] = head['distance']/head['simple_eta']
    head['speed_avg'] = head['slice_id'].map(head.groupby('slice_id')['speed'].mean())
    head['speed_std'] = head['slice_id'].map(head.groupby('slice_id')['speed'].std())
    head['speed_min'] = head['slice_id'].map(head.groupby('slice_id')['speed'].min())
    head['speed_max'] = head['slice_id'].map(head.groupby('slice_id')['speed'].max())
    head['volume'] = head['slice_id'].map(head.groupby('slice_id').size())
    head['old_driver'] = head['driver_id'].map(head.groupby('driver_id').size())
    
    head.set_index('order_id',inplace=True)
########################
###link features
########################
    link = link[link['link_time']>0]
    link['link_eta'] = link['link_time']*link['link_ratio']    
    link['next_link'] = link['link_id'].map(nl)
    link['slice_id'] = link['order_id'].map(head['slice_id'])
    
    gl = link.groupby('order_id')
    head['link_no'] = head.index.map(gl.size())
    head['link_time_sum'] = head.index.map(gl['link_eta'].sum())
    head['link_time_avg'] = head.index.map(gl['link_time'].mean())
    head['link_time_std'] = head.index.map(gl['link_time'].std())
    head['link_time_max'] = head.index.map(gl['link_time'].max())
    head['link_time_min'] = head.index.map(gl['link_time'].min())
    
#     gl = link.groupby(['order_id','link_current_status'])['link_time']
#     head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('link_no_'))
#     head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('link_time_sum_'))
#     head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('link_time_avg_'))
#     head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('link_time_std_'))
#     head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('link_time_max_'))
#     head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('link_time_min_'))

    gl = link.groupby(['order_id','next_link'])['link_time']
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('next_no_'))
    head = head.join(gl.sum().unstack(level=-1,fill_value=0).add_prefix('next_time_sum_'))
    head = head.join(gl.mean().unstack(level=-1,fill_value=0).add_prefix('next_time_avg_'))
    head = head.join(gl.std().unstack(level=-1,fill_value=0).add_prefix('next_time_std_'))
    head = head.join(gl.max().unstack(level=-1,fill_value=0).add_prefix('next_time_max_'))
    head = head.join(gl.min().unstack(level=-1,fill_value=0).add_prefix('next_time_min_'))
    
#     arrival_status = gl['link_arrival_status'].value_counts().unstack(level=-1).fillna(0)
#     head['arrival_0'] = head.index.map(arrival_status[0])
#     head['arrival_1'] = head.index.map(arrival_status[1])
#     head['arrival_2'] = head.index.map(arrival_status[2])
#     head['arrival_3'] = head.index.map(arrival_status[3])
#     head['arrival_4'] = head.index.map(arrival_status[4])

########################
###cross features
########################
    gc = cross.groupby('order_id') 
    head['cross_no'] = head.index.map(gc.size())
    head['cross_sum'] = head.index.map(gc['cross_time'].sum())
    head['cross_avg'] = head.index.map(gc['cross_time'].mean())
    head['cross_std'] = head.index.map(gc['cross_time'].std())
    head['cross_max'] = head.index.map(gc['cross_time'].max())
    head['cross_min'] = head.index.map(gc['cross_time'].min())
#     head['cross_ratio'] = head['cross_sum']/head['simple_eta']

    head = head.fillna(0).reset_index().set_index('slice_id')
    gl = link.groupby(['slice_id','link_current_status'])
    head = head.join(gl.size().unstack(level=-1,fill_value=0).add_prefix('link_status_no_'))
    head = head.fillna(0).reset_index()
    if(task=='train'):
        head['date'] = f[:8]
    return head

In [9]:
network = pd.read_table('/media/fan/hdd/giscup/giscup_2021/nextlinks.txt',sep=' ',names=['link_id','next_link'])
network['next_link_no'] = network['next_link'].apply(lambda z: 4 if len(list(z.split(',')))>4 else len(list(z.split(','))))
nl = network.set_index('link_id')['next_link_no'].astype('int')

In [10]:
TRAIN_FILES = ['202008'+str(i).zfill(2)+'.csv' for i in range(1,32)]
for i,f in tqdm(enumerate(TRAIN_FILES)):    
#     print(f)
    head = pd.read_csv('/media/fan/hdd/giscup/giscup_2021/train_head/'+f)
    link = pd.read_csv('/media/fan/hdd/giscup/giscup_2021/train_link/'+f)
    cross = pd.read_csv('/media/fan/hdd/giscup/giscup_2021/train_cross/'+f)
    if(head.shape[0]==0):
        continue
    head = gen_feats('train',head,link,cross,nl)
    if(i==0):
        head.to_csv('/media/fan/hdd/giscup/train.csv',index=False)
    else:
        head.to_csv('/media/fan/hdd/giscup/train.csv',mode='a',header=False,index=False)
#     sample = sample.append(head,ignore_index=True)
# sample = reduce_mem_usage(sample)
# sample.to_csv('/media/fan/hdd/giscup/train.csv',index=False)

0it [00:00, ?it/s]<ipython-input-8-5018e15a5744>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['link_eta'] = link['link_time']*link['link_ratio']
<ipython-input-8-5018e15a5744>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['next_link'] = link['link_id'].map(nl)
<ipython-input-8-5018e15a5744>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [11]:
head = pd.read_csv('/media/fan/hdd/giscup/giscup_2021/test_head.csv')
link = pd.read_csv('/media/fan/hdd/giscup/giscup_2021/test_link.csv')
cross = pd.read_csv('/media/fan/hdd/giscup/giscup_2021/test_cross.csv')

head = gen_feats('test',head,link,cross,nl)
head['date'] = '20200901'
head.to_csv('/media/fan/hdd/giscup/test.csv',index=False)

<ipython-input-8-5018e15a5744>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['link_eta'] = link['link_time']*link['link_ratio']
<ipython-input-8-5018e15a5744>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['next_link'] = link['link_id'].map(nl)
<ipython-input-8-5018e15a5744>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa